### Import Required Libraries and Set Up Environment Variables

In [41]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import urllib.parse
import re

In [42]:
# Set environment variables from the .env in the local environment

dotenv_path = 'API_Key.env'
load_dotenv(dotenv_path)

True

In [43]:

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
print(nyt_api_key)
print(tmdb_api_key)

kSXfpHnlnkdLFzQKKU6GjXxWmNoCal5U
a44e979259b0e486d35e5ae9342b346d


### Access the New York Times API

In [44]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Initial simpler query to test if we get any results
#filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
# Use simpler queries to debug
filter_queries = [
    'section_name:"Movies"',
    'type_of_material:"Review"',
    'headline:"love"'
]

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Test each filter query individually
for filter_query in filter_queries:
    encoded_filter_query = urllib.parse.quote(filter_query)
    # Build URL
    query_url = (
        f"{url}"
        f"api-key={nyt_api_key}&"
        f"fq={encoded_filter_query}&"
        f"sort={sort}&"
        f"fl={field_list}&"
        f"begin_date={begin_date}&"
        f"end_date={end_date}&"
        f"page=0"
    )
    # Make a request to test the URL
    response = requests.get(query_url)
    print(f"Testing filter query: {filter_query}")
    print(response.url)
    print(f"Page 0 response status code: {response.status_code}")
    print(response.json())
    print("\n")


Testing filter query: section_name:"Movies"
https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=kSXfpHnlnkdLFzQKKU6GjXxWmNoCal5U&fq=section_name%3A%22Movies%22&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&page=0
Page 0 response status code: 200
{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/31/movies/reality-review.html', 'snippet': 'A new docudrama starring Sydney Sweeney as Reality Winner is gripping, even as it strips a true story of its political context.', 'source': 'The New York Times', 'headline': {'main': '‘Reality’ Review: An Unusual Suspect', 'kicker': 'Critic’s Pick', 'content_kicker': None, 'print_headline': 'Examining A Most Unusual Suspect', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name'

In [45]:
    # Make a request to test the URL
    response = requests.get(query_url)
    print(f"Testing filter query: {filter_query}")
    print(response.url)
    print(f"Page 0 response status code: {response.status_code}")
    print(response.json())
    print("\n")

Testing filter query: headline:"love"
https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=kSXfpHnlnkdLFzQKKU6GjXxWmNoCal5U&fq=headline%3A%22love%22&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&page=0
Page 0 response status code: 200
{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/30/style/tiny-modern-love-stories-everyone-knew-but-him.html', 'snippet': 'Modern Love in miniature, featuring reader-submitted stories of no more than 100 words.', 'source': 'The New York Times', 'headline': {'main': 'Tiny Love Stories: ‘Everyone Knew but Him’', 'kicker': 'Modern Love', 'content_kicker': None, 'print_headline': None, 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Love (Emotion)', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Modern Love (Tim

In [46]:
# Create an empty list to store the reviews
reviews_list = []

In [47]:

# loop through pages 0-19
for page in range(0, 20):
    query_url = f"{url}api-key={nyt_api_key}&q={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&page={page}"
   
    # create query with a page number
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    reviews = response.json()  
# Print the response status code and a part of the response content for debugging
    print(f"Page {page} response status code: {response.status_code}")
    if response.status_code != 200:
        print(f"Error: {response.json()}")
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    # loop through the reviews["response"]["docs"] and append each review to the list
    # Print the page that was just retrieved
    # Print the page number that had no results then break from the loop  
    try:
        docs = reviews.get("response", {}).get("docs", [])
        if not docs:
            print(f"No results found on page {page}")
            break
        for doc in docs:
            reviews_list.append(doc)
        print(f"Page {page} processed")
    except Exception as e:
        print(f"An error occurred on page {page}: {e}")
        break

    



Page 0 response status code: 200
Page 0 processed
Page 1 response status code: 200
Page 1 processed
Page 2 response status code: 200
Page 2 processed
Page 3 response status code: 200
Page 3 processed
Page 4 response status code: 200
Page 4 processed
Page 5 response status code: 200
Page 5 processed
Page 6 response status code: 200
Page 6 processed
Page 7 response status code: 200
Page 7 processed
Page 8 response status code: 200
Page 8 processed
Page 9 response status code: 200
Page 9 processed
Page 10 response status code: 200
Page 10 processed
Page 11 response status code: 200
Page 11 processed
Page 12 response status code: 200
Page 12 processed
Page 13 response status code: 200
Page 13 processed
Page 14 response status code: 200
Page 14 processed
Page 15 response status code: 200
Page 15 processed
Page 16 response status code: 200
Page 16 processed
Page 17 response status code: 200
Page 17 processed
Page 18 response status code: 200
Page 18 processed
Page 19 response status code: 20

In [48]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/31/arts/pride-month-events-nyc.html",
        "snippet": "Indulge in gay nostalgia with Christina Aguilera and Junior Vasquez, see Billy Porter march, or dance the night away at Body Hack.",
        "source": "The New York Times",
        "headline": {
            "main": "Pride Month in New York Is Back. Here\u2019s How to Celebrate.",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Plenty of Events to Celebrate Strength and Solidarity",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Homosexuality and Bisexuality",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "subject",
                "value": "LGBTQ Pride Month (US)",
                "rank": 2,
                "major": "N"
        

In [49]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df = pd.json_normalize(reviews_list)
df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/31/arts/pride-...,Indulge in gay nostalgia with Christina Aguile...,The New York Times,"[{'name': 'subject', 'value': 'Homosexuality a...",2023-05-31T18:52:55+0000,1103,Pride Month in New York Is Back. Here’s How to...,None,None,Plenty of Events to Celebrate Strength and Sol...,None,None,None,By Erik Piepenburg,"[{'firstname': 'Erik', 'middlename': None, 'la...",None
1,https://www.nytimes.com/2023/05/31/arts/music/...,"After a triumphant house debut in 2019, the ba...",The New York Times,"[{'name': 'subject', 'value': 'Opera', 'rank':...",2023-05-31T17:38:32+0000,585,Review: A Singer Returns to Shore at the Met i...,Critic’s Pick,None,"Rising and Raging, a Bass-Baritone Returns to ...",None,None,None,By Oussama Zahr,"[{'firstname': 'Oussama', 'middlename': None, ...",None
2,https://www.nytimes.com/2023/05/31/nyregion/pi...,"Penny, the unofficial mascot of Engine Company...",The New York Times,"[{'name': 'glocations', 'value': 'New York Cit...",2023-05-31T04:45:05+0000,1318,Who Needs a Dalmatian? This Firehouse Has a Pig.,New York Today,None,None,None,None,None,By James Barron,"[{'firstname': 'James', 'middlename': None, 'l...",None
3,https://www.nytimes.com/2023/05/30/sports/tenn...,Women’s tennis was headlined for more than two...,The New York Times,"[{'name': 'subject', 'value': 'Tennis', 'rank'...",2023-05-30T22:06:57+0000,1086,Can These Russians Become Tennis’s Next Great ...,None,None,Russian Teenagers Could Provide The Sport’s Ne...,None,None,None,By Matthew Futterman,"[{'firstname': 'Matthew', 'middlename': None, ...",None
4,https://www.nytimes.com/2023/05/29/world/asia/...,After a painting by the Dutch artist sold at a...,The New York Times,"[{'name': 'persons', 'value': 'van Gogh, Vince...",2023-05-29T09:01:28+0000,2521,The Mystery of the Disappearing van Gogh,None,None,The Vanishing van Gogh: A $62 Million Mystery,None,None,None,"By Michael Forsythe, Isabelle Qian, Muyi Xiao ...","[{'firstname': 'Michael', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2023/01/20/briefing/de...,A political fight is again putting the economy...,The New York Times,"[{'name': 'subject', 'value': 'internal-storyl...",2023-01-20T11:47:43+0000,1724,Routine Crisis,None,None,None,None,None,None,By German Lopez,"[{'firstname': 'German', 'middlename': None, '...",None
196,https://www.nytimes.com/2023/01/19/arts/music/...,He was an original member of the Byrds and a f...,The New York Times,"[{'name': 'persons', 'value': 'Crosby, David',...",2023-01-20T00:08:26+0000,2392,"David Crosby, Folk-Rock Voice of the 1960s Who...",None,None,"David Crosby, Who Founded Two Classic Rock Ban...",None,None,None,By Jim Farber,"[{'firstname': 'Jim', 'middlename': None, 'las...",None
197,https://www.nytimes.com/2023/01/18/learning/wh...,"24 comics, paintings, photos, drawings and col...",The New York Times,[],2023-01-19T00:04:33+0000,4434,What It’s Like to Be a Teenager Now: The Winne...,Contests,None,None,None,None,None,By The Learning Network,[],The Learning Network
198,https://www.nytimes.com/2023/01/18/business/ca...,"As president of Volkswagen of America, he push...",The New York Times,"[{'name': 'subject', 'value': 'Automobiles', '...",2023-01-18T20:27:22+0000,915,Carl Hahn Dies at 96; Made the VW Beetle Ubiqu...,None,None,"Carl H. Hahn, 96, Who Turned a Vehicle Into a ...",None,None,None,By Sam Roberts,"[{'firstname': 'Sam', 'middlename': None, 'las...",None


In [50]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
def extract_title(headline):
    match = re.search(r'\u2018(.*?)\u2019', headline)
    if match:
        title = match.group(1) + " Review"
        return title
    else:
        return None
    
df['title'] = df['headline.main'].apply(extract_title)


df.head()  

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/31/arts/pride-...,Indulge in gay nostalgia with Christina Aguile...,The New York Times,"[{'name': 'subject', 'value': 'Homosexuality a...",2023-05-31T18:52:55+0000,1103,Pride Month in New York Is Back. Here’s How to...,None,None,Plenty of Events to Celebrate Strength and Sol...,None,None,None,By Erik Piepenburg,"[{'firstname': 'Erik', 'middlename': None, 'la...",None,None
1,https://www.nytimes.com/2023/05/31/arts/music/...,"After a triumphant house debut in 2019, the ba...",The New York Times,"[{'name': 'subject', 'value': 'Opera', 'rank':...",2023-05-31T17:38:32+0000,585,Review: A Singer Returns to Shore at the Met i...,Critic’s Pick,None,"Rising and Raging, a Bass-Baritone Returns to ...",None,None,None,By Oussama Zahr,"[{'firstname': 'Oussama', 'middlename': None, ...",None,Dutchman Review
2,https://www.nytimes.com/2023/05/31/nyregion/pi...,"Penny, the unofficial mascot of Engine Company...",The New York Times,"[{'name': 'glocations', 'value': 'New York Cit...",2023-05-31T04:45:05+0000,1318,Who Needs a Dalmatian? This Firehouse Has a Pig.,New York Today,None,None,None,None,None,By James Barron,"[{'firstname': 'James', 'middlename': None, 'l...",None,None
3,https://www.nytimes.com/2023/05/30/sports/tenn...,Women’s tennis was headlined for more than two...,The New York Times,"[{'name': 'subject', 'value': 'Tennis', 'rank'...",2023-05-30T22:06:57+0000,1086,Can These Russians Become Tennis’s Next Great ...,None,None,Russian Teenagers Could Provide The Sport’s Ne...,None,None,None,By Matthew Futterman,"[{'firstname': 'Matthew', 'middlename': None, ...",None,None
4,https://www.nytimes.com/2023/05/29/world/asia/...,After a painting by the Dutch artist sold at a...,The New York Times,"[{'name': 'persons', 'value': 'van Gogh, Vince...",2023-05-29T09:01:28+0000,2521,The Mystery of the Disappearing van Gogh,None,None,The Vanishing van Gogh: A $62 Million Mystery,None,None,None,"By Michael Forsythe, Isabelle Qian, Muyi Xiao ...","[{'firstname': 'Michael', 'middlename': None, ...",None,None


In [52]:
# Extract 'name' and 'value' from items in "keywords" column
# def extract_keywords(keyword_list):
#     extracted_keywords = ""
#     for item in keyword_list:
#         # Extract 'name' and 'value'
#         keyword = f"{item['name']}: {item['value']};" 
#         # Append the keyword item to the extracted_keywords list
#         extracted_keywords += keyword
#     return extracted_keywords

# # Fix the "keywords" column by converting cells from a list to a string
# df['keywords'] = df['keywords'].apply(extract_keywords)
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        if isinstance(item, dict):
            keyword = f"{item.get('name', '')}: {item.get('value', '')}; "
            extracted_keywords += keyword
    return extracted_keywords.strip()  # Strip trailing space

# Fix the "keywords" column
df['keywords'] = df['keywords'].apply(extract_keywords)

In [53]:
df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/31/arts/pride-...,Indulge in gay nostalgia with Christina Aguile...,The New York Times,,2023-05-31T18:52:55+0000,1103,Pride Month in New York Is Back. Here’s How to...,None,None,Plenty of Events to Celebrate Strength and Sol...,None,None,None,By Erik Piepenburg,"[{'firstname': 'Erik', 'middlename': None, 'la...",None,None
1,https://www.nytimes.com/2023/05/31/arts/music/...,"After a triumphant house debut in 2019, the ba...",The New York Times,,2023-05-31T17:38:32+0000,585,Review: A Singer Returns to Shore at the Met i...,Critic’s Pick,None,"Rising and Raging, a Bass-Baritone Returns to ...",None,None,None,By Oussama Zahr,"[{'firstname': 'Oussama', 'middlename': None, ...",None,Dutchman Review
2,https://www.nytimes.com/2023/05/31/nyregion/pi...,"Penny, the unofficial mascot of Engine Company...",The New York Times,,2023-05-31T04:45:05+0000,1318,Who Needs a Dalmatian? This Firehouse Has a Pig.,New York Today,None,None,None,None,None,By James Barron,"[{'firstname': 'James', 'middlename': None, 'l...",None,None
3,https://www.nytimes.com/2023/05/30/sports/tenn...,Women’s tennis was headlined for more than two...,The New York Times,,2023-05-30T22:06:57+0000,1086,Can These Russians Become Tennis’s Next Great ...,None,None,Russian Teenagers Could Provide The Sport’s Ne...,None,None,None,By Matthew Futterman,"[{'firstname': 'Matthew', 'middlename': None, ...",None,None
4,https://www.nytimes.com/2023/05/29/world/asia/...,After a painting by the Dutch artist sold at a...,The New York Times,,2023-05-29T09:01:28+0000,2521,The Mystery of the Disappearing van Gogh,None,None,The Vanishing van Gogh: A $62 Million Mystery,None,None,None,"By Michael Forsythe, Isabelle Qian, Muyi Xiao ...","[{'firstname': 'Michael', 'middlename': None, ...",None,None


In [54]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
nyt_titles = df['title'].tolist()
print(nyt_titles)

[None, 'Dutchman Review', None, None, None, None, 'Succession Review', None, None, None, 'The Inheritance Review', None, None, None, None, None, None, None, None, None, 'The Diplomat Review', None, None, None, None, None, None, None, None, 'The Sign in Sidney Brustein Review', 'A Nightmare Review', None, None, 'Superfake Review', None, 'Ted Lasso, Review', None, None, None, None, None, 'Take It or Leave It Review', None, None, None, 'Birnam Wood Review', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'God and Faith Review', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Yellowjackets Review', None, None, None, None, None, None, 'Perry Mason Review', None, 'Succession Review', 'Bad Cinderella Review', None, 'The Daily Show Review', None, None, None, None, None, None, None, None, None, None, None, None, None

In [55]:
# # Filter out None values and remove " Review" suffix
cleaned_titles = [title.replace(" Review", "") for title in nyt_titles if title is not None]

# # Print the cleaned titles
print(cleaned_titles)

['Dutchman', 'Succession', 'The Inheritance', 'The Diplomat', 'The Sign in Sidney Brustein', 'A Nightmare', 'Superfake', 'Ted Lasso,', 'Take It or Leave It', 'Birnam Wood', 'God and Faith', 'Yellowjackets', 'Perry Mason', 'Succession', 'Bad Cinderella', 'The Daily Show', 'The Great British Bake Off Musical', 'Vanderpump Rules', 'True Lies', 'Is Not Normal and Should Not Exist', 'Party Down', 'Love Trip: Paris', 'Bird?', 'Midsize', 'The Last of Us', 'Money', 'Wise Selfishness']


### Access The Movie Database API

In [56]:
# Prepare The Movie Database query
tmdb_url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
tmdb_url

'https://api.themoviedb.org/3/search/movie?query='

In [57]:
# Create an empty list to store the results
tmdb_list = []

In [58]:
# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0


In [59]:
# Loop through the titles
for title in cleaned_titles:
    try:
        tmdb_query_url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={title}"
     # Perform a "GET" request for The Movie Database
        tmdb_response = requests.get(tmdb_query_url)
            # Add 1 to the request counter
    # Make the request to TMDB
   
        request_counter += 1
# Check if we need to sleep before making a request
        if request_counter % 50 == 0:
            print(f"API rate limit - (Request {request_counter})")
            time.sleep(10)
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
        if tmdb_response.status_code == 200:
            tmdb_list.append(tmdb_response.json())
        else: 
            print(f"No movie found for title {title}")
    except Exception as e:
        print(f"An error occurred searching for: {title}")
        print(f"Error details: {str(e)}")

In [60]:
tmdb_list

[{'page': 1,
  'results': [{'adult': False,
    'backdrop_path': '/bb7DAbjQvFebdNn3Hy3bmhenKhG.jpg',
    'genre_ids': [18],
    'id': 66770,
    'original_language': 'en',
    'original_title': 'Dutchman',
    'overview': "A young conservative black man, minding his own business, rides a nearly empty subway car. The only other passenger, a blonde vixen looking for trouble, sizes him up. Sexual tension, racial bigotry and righteous fury collide in a razor's edge confrontation between this unlikely pair.",
    'popularity': 2.24,
    'poster_path': '/tQGSuTQO8TMSLLgqgAWaCTAuv6i.jpg',
    'release_date': '1966-12-28',
    'title': 'Dutchman',
    'video': False,
    'vote_average': 4.6,
    'vote_count': 12},
   {'adult': False,
    'backdrop_path': None,
    'genre_ids': [],
    'id': 398061,
    'original_language': 'en',
    'original_title': 'The Dutchman and Snake',
    'overview': '"Do not bite the hand that feed you" - African Proverb',
    'popularity': 0.001,
    'poster_path': N

In [61]:

        # Get movie id
        if tmdb_response.status_code == 200:
            tmdb_data = tmdb_response.json()

            # Check if the movie was found
            if tmdb_data['results']:
                movie_id = tmdb_data['results'][0]['id']
                # Make a request for the full movie details
                tmdb_movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
                tmdb_movie_response = requests.get(tmdb_movie_url)

                if tmdb_movie_response.status_code == 200:
                    movie_details = tmdb_movie_response.json()

                    # Extract the genre names into a list
                    genre_names = [genre['name'] for genre in movie_details.get('genres', [])]

                    # Extract the spoken_languages' English name into a list
                    language_names = [language['english_name'] for language in movie_details.get('spoken_languages', [])]

                    # Extract the production_countries' name into a list
                    country_names = [country['name'] for country in movie_details.get('production_countries', [])]

                    # Add the relevant data to a dictionary and append it to the tmdb_list list
                    movie_dict = {
                        'title': movie_details.get('title'),
                        'genres': genre_names,
                        'spoken_languages': language_names,
                        'production_countries': country_names,
                        'release_date': movie_details.get('release_date'),
                        'runtime': movie_details.get('runtime'),
                        'vote_average': movie_details.get('vote_average'),
                        'overview': movie_details.get('overview')
                    }

                    # Append the movie details to the list
                    tmdb_list.append(movie_dict)

                    # Print out the title that was found
                    print(f"Found details for movie: {title}")
                else:
                    print(f"Failed to get details for movie ID {movie_id}")
            else:
                print(f"No movie found for title {title}")
        else:
            print(f"Failed to search for movie: {title} - Status code: {tmdb_response.status_code}")


No movie found for title Wise Selfishness


In [65]:
movie_id

NameError: name 'movie_id' is not defined

In [62]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_list[:5], indent=4))


[
    {
        "page": 1,
        "results": [
            {
                "adult": false,
                "backdrop_path": "/bb7DAbjQvFebdNn3Hy3bmhenKhG.jpg",
                "genre_ids": [
                    18
                ],
                "id": 66770,
                "original_language": "en",
                "original_title": "Dutchman",
                "overview": "A young conservative black man, minding his own business, rides a nearly empty subway car. The only other passenger, a blonde vixen looking for trouble, sizes him up. Sexual tension, racial bigotry and righteous fury collide in a razor's edge confrontation between this unlikely pair.",
                "popularity": 2.24,
                "poster_path": "/tQGSuTQO8TMSLLgqgAWaCTAuv6i.jpg",
                "release_date": "1966-12-28",
                "title": "Dutchman",
                "video": false,
                "vote_average": 4.6,
                "vote_count": 12
            },
            {
             

In [63]:
# Convert the results to a DataFrame
tmdb_df = pd.json_normalize(tmdb_list)

### Merge and Clean the Data for Export

In [66]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df, tmdb_df, on='title', how='inner')


KeyError: 'title'

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing

# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame
print(merged_df.head())

In [ ]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=['byline.person'])

In [ ]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('merged_movie_data.csv', index=False)